by Jessica

In [3]:
df1 = pd.read_csv("C:\\Users\\Jessica Tran\\Downloads\\2017_GSM_scaled.csv")
df1 = df1[['approx_price_eur_scale', 'battery_mah_scale','ram_in_gb_scale','internal_memory_in_gb_scale','model','brand']]
df1.shape

(8359, 6)

In [7]:
import pandas as pd
df2 = pd.read_csv("C:\\Users\\Jessica Tran\\Downloads\\World's Best-Selling Phone's Sales\\top 120 best-selling mobile phones.csv")
df2 = df2[['Manufacturer', 'Model','Units Sold (million )','Rank']]
df2.head()

,Manufacturer,Model,Units Sold (million ),Rank
0,Nokia,1100,250.0,1
1,Nokia,1110,247.5,2
2,Apple,iPhone 6 & 6 Plus,222.4,3
3,Nokia,105 Series,200.0,4
4,Apple,iPhone 6S & 6S Plus,174.1,5


In [4]:
def fuzzy_match_and_join(df1, df2, key1, key2, threshold, limit=1):
    """
    Perform fuzzy matching and join two datasets on model names.

    Args:
        df1: First DataFrame
        df2: Second DataFrame
        key1: Column name in df1 for matching
        key2: Column name in df2 for matching
        threshold: Minimum similarity score to consider a match.
        limit: Number of best matches to return per row.

    Returns:
        Merged DataFrame with fuzzy-matched keys.
    """
    # Create a dictionary to store matches
    matches = []
    for model in df1[key1]:
        best_matches = process.extract(
            model, df2[key2], scorer=fuzz.ratio, limit=limit
        )
        for match, score, idx in best_matches:
            if score >= threshold:
                matches.append({
                    key1: model,
                    key2: match,
                    'similarity_score': score,
                    'index_df2': idx
                })
    
    # Create a DataFrame of matches
    match_df = pd.DataFrame(matches)
    
    # Merge the original DataFrames
    df2_with_index = df2.reset_index().rename(columns={"index": "index_df2"})
    merged_df = (
        df1.merge(match_df, on=key1)
           .merge(df2_with_index, left_on='index_df2', right_on='index_df2', suffixes=('_df1', '_df2'))
           .drop(columns=['index_df2'])
    )
    return merged_df

In [10]:
from rapidfuzz import process, fuzz

merged_df = fuzzy_match_and_join(df2, df1, key1='Model', key2='model', threshold=90)
merged_df = pd.DataFrame(data=merged_df)
merged_df

,Manufacturer,Model,Units Sold (million ),Rank,model_df1,similarity_score,approx_price_eur_scale,battery_mah_scale,ram_in_gb_scale,internal_memory_in_gb_scale,model_df2,brand
0,Nokia,1100,250.0,1,1100,100.000000,NaN,-0.750299,NaN,NaN,1100,Nokia
1,Nokia,1110,247.5,2,1110,100.000000,-0.276401,-0.705810,NaN,-0.448724,1110,Nokia
2,Apple,iPhone 5S,164.5,6,iPhone 5,94.117647,0.256900,-0.225336,-0.078241,2.570061,iPhone 5,Apple
3,Apple,iPhone5,146.2,15,iPhone 5,93.333333,0.256900,-0.225336,-0.078241,2.570061,iPhone 5,Apple
4,Nokia,3210,161.0,7,3210,100.000000,NaN,NaN,NaN,NaN,3210,Nokia
5,Nokia,6600,150.0,11,6600,100.000000,NaN,-0.750299,NaN,-0.448632,6600,Nokia
6,Nokia,1200,150.0,12,1200,100.000000,-0.294178,-0.883763,NaN,-0.448724,1200,Nokia
7,Nokia,5230,150.0,13,5230,100.000000,NaN,-0.438880,NaN,-0.446705,5230,i-mobile
8,Samsung,E1100,150.0,14,E1100,100.000000,-0.258624,-0.839275,NaN,-0.448862,E1100,Samsung
9,Motorola,RAZRV3,130.0,17,RAZR V3,92.307692,-0.169741,-0.901559,NaN,-0.448678,RAZR V3,Motorola
